In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
#wikipedia link: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
from bs4 import BeautifulSoup # this module helps in web scrapping.


url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text
soup = BeautifulSoup(data, 'html5lib')


In [3]:
table_contents=[]
table=soup.findAll('table')[0]
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

<h2>Create a dataframe from List & Cleaning up Borough column

In [4]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

<h2>Looking at all rows to make sure data looks clean (esp. the Borough)

In [5]:
#toggle this on/off to see all rows
#pd.set_option("display.max_rows",  None)

df.head() #only see the first 5


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [6]:
df.shape

(103, 3)

<h2>Adding Longitude and Latitude

In [7]:
#Getting the files from csv file in my github since geocoder was unreliable
url2 = "https://raw.githubusercontent.com/aymiee/Toronto/master/Geospatial_Coordinates.csv"
#s=requests.get(url2).content
dfCoordinates = pd.read_csv(url2)
#print(dfCoordinates.loc[dfCoordinates['Postal Code'] == 'M3A'].Latitude)

df['Latitude'] = None
df['Longitude'] = None

for index, row in df.iterrows():  
    Latitude = dfCoordinates.loc[dfCoordinates['Postal Code'] == row['PostalCode']].Latitude
    Longitude = dfCoordinates.loc[dfCoordinates['Postal Code'] == row['PostalCode']].Longitude
    row['Latitude'] = float(Latitude)
    row['Longitude'] = float(Longitude)
    
df


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<h2>Segmenting and Clustering Neighborhoods in Toronto
<h2>Focusing on Etobicoke Neighborhood

In [8]:
neighborhoods = df.loc[df['Borough'] == "Etobicoke"]
neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
17,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
70,M9P,Etobicoke,Westmount,43.696319,-79.532242
77,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
88,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
93,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 1 boroughs and 11 neighborhoods.


In [88]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


# Create a map of Toronto with neighborhoods  (Etobioke) superimposed on top.

In [12]:
import folium # map rendering library
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    #print(label)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto


<!-- linking because can't show maps in github -->
<img src = "https://raw.githubusercontent.com/aymiee/Toronto/master/mapoftoronto.PNG" />

In [91]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


# Let's explore the Islington Avenue neighborhood in our dataframe.


In [17]:
neighborhoods.loc[102, 'Neighborhood']

'Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West'

In [18]:
neighborhood_latitude = neighborhoods.loc[102, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[102, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[102, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West are 43.6288408, -79.5209994.


# 1 Now, let's get the top 100 venues that are in Islington Avenue within a radius of 500 meters.

In [19]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius



url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=EKCLZKFATGR4JMIDQ5WXTRGHB2T5D5NCF3MPMJ2G14AK23KG&client_secret=K2WFSPSDQ45BSXZVOAYCQC4WDYKQDIEQQEUHAMOXOQ0ZXBIV&v=20180605&ll=43.6288408,-79.5209994&radius=500&limit=100'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '606e225b45de7f3bb2d4869c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Islington - City Centre West',
  'headerFullLocation': 'Islington - City Centre West, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 15,
  'suggestedBounds': {'ne': {'lat': 43.6333408045, 'lng': -79.51479402615581},
   'sw': {'lat': 43.6243407955, 'lng': -79.52720477384418}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b5fcadff964a520eecc29e3',
       'name': 'Wingporium',
       'location': {'address': '1000 Islington Ave',
        'crossStreet': 'at Titan Rd',
        'lat': 43.630275355081025,
        'lng': -79.51816911632163,
       

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
     
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON
#nearby_venues.head()

In [23]:
venues = results['response']['groups'][0]['items']

    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
#nearby_venues.apply(get_category_type, axis=1)

for index, row in nearby_venues.iterrows(): 
    nearby_venues['venue.categories'] = get_category_type(row)
    
    # clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wingporium,Burrito Place,43.630275,-79.518169
1,South St. Burger,Burrito Place,43.631314,-79.518408
2,Dollarama,Burrito Place,43.629883,-79.518627
3,Healthy Planet,Burrito Place,43.630214,-79.518495
4,Artisano Bakery Café,Burrito Place,43.631006,-79.518172


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

15 venues were returned by Foursquare.


# 2. Explore Neighborhoods in Toronto

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h2>Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [26]:
# type your answer here
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Islington Avenue
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Westmount
Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens
New Toronto, Mimico South, Humber Bay Shores
South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens
Alderwood, Long Branch
The Kingsway, Montgomery Road, Old Mill North
Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East
Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West


<h2>Let's check the size of the resulting dataframe

In [27]:
print(toronto_venues.shape)
toronto_venues.head()


(71, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,Canadian Appliance Source Etobicoke,43.649864,-79.558461,Home Service
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,LCBO,43.642099,-79.576592,Liquor Store
2,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,Starbucks,43.641312,-79.576924,Coffee Shop
3,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,The Beer Store,43.641313,-79.576925,Beer Store
4,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,Shoppers Drug Mart,43.641312,-79.576924,Pharmacy


<h2>Let's check how many venues were returned for each neighborhood

In [28]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alderwood, Long Branch",9,9,9,9,9,9
"Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood",8,8,8,8,8,8
"Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens",4,4,4,4,4,4
"Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West",15,15,15,15,15,15
"New Toronto, Mimico South, Humber Bay Shores",12,12,12,12,12,12
"Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East",2,2,2,2,2,2
"South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens",10,10,10,10,10,10
"The Kingsway, Montgomery Road, Old Mill North",2,2,2,2,2,2
"West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale",1,1,1,1,1,1


<h2>Let's find out how many unique categories can be curated from all the returned venues

In [29]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 42 uniques categories.


# 3. Analyze Each Neighborhood

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,Bakery,Baseball Field,Beer Store,Breakfast Spot,Burger Joint,Burrito Place,Bus Line,Business Service,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Dance Studio,Discount Store,Fast Food Restaurant,Fried Chicken Joint,Grocery Store,Gym,Hardware Store,Home Service,Intersection,Kids Store,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Restaurant,River,Sandwich Place,Skating Rink,Supplement Shop,Tanning Salon,Thrift / Vintage Store,Video Store,Wings Joint
0,"West Deane Park, Princess Gardens, Martin Grov...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


<h2>And let's examine the new dataframe size.

In [31]:
toronto_onehot.shape

(71, 43)

<h2>Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [32]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Bakery,Baseball Field,Beer Store,Breakfast Spot,Burger Joint,Burrito Place,Bus Line,Business Service,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Dance Studio,Discount Store,Fast Food Restaurant,Fried Chicken Joint,Grocery Store,Gym,Hardware Store,Home Service,Intersection,Kids Store,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Restaurant,River,Sandwich Place,Skating Rink,Supplement Shop,Tanning Salon,Thrift / Vintage Store,Video Store,Wings Joint
0,"Alderwood, Long Branch",0.000000,0.000000,0.0,0.000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.111111,0.000000,0.111111,0.000000,0.000000,0.0,0.000000,0.111111,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000,0.00,0.000,0.000,0.111111,0.222222,0.000,0.111111,0.000000,0.0,0.111111,0.111111,0.000000,0.000000,0.000000,0.0,0.000000
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.000000,0.000000,0.0,0.125,0.0,0.000000,0.000000,0.00,0.000000,0.125000,0.000,0.125000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.125000,0.000000,0.000,0.00,0.125,0.125,0.125000,0.125000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,"Kingsview Village, St. Phillips, Martin Grove ...",0.000000,0.000000,0.0,0.000,0.0,0.000000,0.000000,0.25,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000,0.25,0.250,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,"Mimico NW, The Queensway West, South of Bloor,...",0.000000,0.066667,0.0,0.000,0.0,0.066667,0.066667,0.00,0.000000,0.000000,0.000,0.000000,0.066667,0.000000,0.066667,0.066667,0.0,0.066667,0.066667,0.066667,0.0,0.000,0.066667,0.000000,0.000000,0.000,0.00,0.000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.066667,0.000000,0.066667,0.066667,0.066667,0.0,0.066667
4,"New Toronto, Mimico South, Humber Bay Shores",0.083333,0.083333,0.0,0.000,0.0,0.000000,0.000000,0.00,0.083333,0.083333,0.000,0.083333,0.000000,0.000000,0.000000,0.083333,0.0,0.000000,0.083333,0.000000,0.0,0.000,0.000000,0.083333,0.083333,0.000,0.00,0.000,0.000,0.083333,0.083333,0.000,0.000000,0.083333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,"Old Mill South, King's Mill Park, Sunnylea, Hu...",0.000000,0.000000,0.5,0.000,0.5,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000,0.00,0.000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,"South Steeles, Silverstone, Humbergate, Jamest...",0.000000,0.000000,0.0,0.100,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.100000,0.1,0.200000,0.000000,0.000000,0.0,0.000,0.000000,0.100000,0.000000,0.000,0.00,0.000,0.000,0.100000,0.100000,0.000,0.000000,0.000000,0.0,0.100000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000
7,"The Kingsway, Montgomery Road, Old Mill North",0.000000,0.000000,0.0,0.000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000,0.00,0.500,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,"West Deane Park, Princess Gardens, Martin Grov...",0.000000,0.000000,0.0,0.000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.0,0.000,0.000000,0.000000,0.000000,0.000,0.00,0.000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,Westmount,0.000000,0.000000,0.0,0.000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.125,0.125000,0.

<h2>Let's confirm the new size

In [33]:
toronto_grouped.shape

(10, 43)

<h2> Let's print each neighborhood along with the top 5 most common venues

In [34]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.22
1   Coffee Shop  0.11
2  Dance Studio  0.11
3           Gym  0.11
4  Skating Rink  0.11


----Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood----
         venue  freq
0  Coffee Shop  0.12
1   Beer Store  0.12
2         Park  0.12
3    Pet Store  0.12
4     Pharmacy  0.12


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                 venue  freq
0    Mobile Phone Shop  0.25
1                 Park  0.25
2             Bus Line  0.25
3       Sandwich Place  0.25
4  American Restaurant  0.00


----Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West----
                  venue  freq
0           Wings Joint  0.07
1                   Gym  0.07
2        Hardware Store  0.07
3  Fast Food Restaurant  0.07
4        Discount Store  0.07


----New Toronto, Mimico South, Humber Bay Shores----
                 venue  freq
0  American Re

<h2>Let's put that into a pandas dataframe

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<h2>Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Dance Studio,Gym,Skating Rink,Sandwich Place,Pharmacy,Pub,Middle Eastern Restaurant,Mobile Phone Shop
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Coffee Shop,Beer Store,Park,Pet Store,Pharmacy,Pizza Place,Café,Liquor Store,Pub,Middle Eastern Restaurant
2,"Kingsview Village, St. Phillips, Martin Grove ...",Mobile Phone Shop,Park,Bus Line,Sandwich Place,American Restaurant,Playground,Mexican Restaurant,Middle Eastern Restaurant,Pet Store,Pharmacy
3,"Mimico NW, The Queensway West, South of Bloor,...",Wings Joint,Gym,Hardware Store,Fast Food Restaurant,Discount Store,Convenience Store,Bakery,Kids Store,Sandwich Place,Supplement Shop
4,"New Toronto, Mimico South, Humber Bay Shores",American Restaurant,Café,Liquor Store,Bakery,Gym,Pharmacy,Pizza Place,Fast Food Restaurant,Coffee Shop,Restaurant


# 4. Cluster Neighborhoods


<h2>Run k-means to cluster the neighborhood into 5 clusters.

In [37]:
# import k-means from clustering stage
import numpy as np
import pandas as pd # library for data analsysis
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans


In [53]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 3, 2, 2, 0, 2, 4, 1, 2], dtype=int32)

<h2>Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [66]:
# add clustering labels
#kmeans.labels_
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.dropna(subset = ['Cluster Labels'], inplace=True)

toronto_merged.tail() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,2.0,Grocery Store,Pharmacy,Video Store,Beer Store,Sandwich Place,Fried Chicken Joint,Liquor Store,Fast Food Restaurant,Pizza Place,American Restaurant
93,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,2.0,Pizza Place,Coffee Shop,Dance Studio,Gym,Skating Rink,Sandwich Place,Pharmacy,Pub,Middle Eastern Restaurant,Mobile Phone Shop
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,4.0,Park,River,American Restaurant,Playground,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Pet Store,Pharmacy
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,0.0,Baseball Field,Breakfast Spot,American Restaurant,Pub,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pet Store,Pharmacy
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,2.0,Wings Joint,Gym,Hardware Store,Fast Food Restaurant,Discount Store,Convenience Store,Bakery,Kids Store,Sandwich Place,Supplement Shop


<H2>Finally, let's visualize the resulting clusters

In [93]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    #print( cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
       fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

<!-- linking because can't show maps in github -->
<img src = "https://raw.githubusercontent.com/aymiee/Toronto/master/mapcluster.PNG" />

# 5. Examine Clusters

In [83]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
101,Etobicoke,0.0,Baseball Field,Breakfast Spot,American Restaurant,Pub,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pet Store,Pharmacy


In [84]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,1.0,Home Service,Playground,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pet Store,Pharmacy,Pizza Place


In [85]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Etobicoke,2.0,Coffee Shop,Beer Store,Park,Pet Store,Pharmacy,Pizza Place,Café,Liquor Store,Pub,Middle Eastern Restaurant
70,Etobicoke,2.0,Intersection,Sandwich Place,Middle Eastern Restaurant,Pizza Place,Discount Store,Playground,Chinese Restaurant,Coffee Shop,River,Restaurant
88,Etobicoke,2.0,American Restaurant,Café,Liquor Store,Bakery,Gym,Pharmacy,Pizza Place,Fast Food Restaurant,Coffee Shop,Restaurant
89,Etobicoke,2.0,Grocery Store,Pharmacy,Video Store,Beer Store,Sandwich Place,Fried Chicken Joint,Liquor Store,Fast Food Restaurant,Pizza Place,American Restaurant
93,Etobicoke,2.0,Pizza Place,Coffee Shop,Dance Studio,Gym,Skating Rink,Sandwich Place,Pharmacy,Pub,Middle Eastern Restaurant,Mobile Phone Shop
102,Etobicoke,2.0,Wings Joint,Gym,Hardware Store,Fast Food Restaurant,Discount Store,Convenience Store,Bakery,Kids Store,Sandwich Place,Supplement Shop


In [86]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
77,Etobicoke,3.0,Mobile Phone Shop,Park,Bus Line,Sandwich Place,American Restaurant,Playground,Mexican Restaurant,Middle Eastern Restaurant,Pet Store,Pharmacy


In [87]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
98,Etobicoke,4.0,Park,River,American Restaurant,Playground,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Pet Store,Pharmacy
